# Got a new job so stopped working on this

I took a different angle if it helps anyone:
Find strongest point in 5 second sample
Normalize all arrays to have the max peak at the same point (to ease comparison)
focus on +/- 0.5 sec around that peak
Fingerprint the frequencies leading up to and after the peak and the intensities.

I was looking to identify the region of birds for each of the 3 areas (say N. California), which would then limit the range of possible birds to choose from.

Key differences I noticed in the data:
slope leading to peak and away tended to be relatively unique
frequencies on average for the peak can be used to reduce the possible birds
I normalized the frequencies measured to be in the range 1800-3600 by multipllying and dividing, as they would have resonances in that range and again it simplified comparing them.

(then I got a new job and abandoned for the last month...)

Maybe this will help give some new ideas, Mark

==================

Other features:
Threading and Que Operations
Using sample data for submissions
Low Pass and High Pass filter functions
FFT with graphing
North America Only function (as that is what 3 regions are specified in)



# Hot Ideas

1.  Fingerprint, ideal steps
    * Look for Max Magnitude (power) in first 3 min of song
    * Start analysis say 1/10 second earlier if possible
    * Find peak freqencies from there in steps as fingerprint (having way of capturing delay gaps)
    * FP may look like: F: 1900,2015,1890, 1700, ..10, FP: 1900,3,x,2015,3,x...etc.
    * Then scan at lower power for second bird if it exists, or bandgap our primary frequency of bird 1 and rebuild song backwards
2.  Try getting all the data with only gaps and no FFT, ideally with more clever gap measurements (with a peak detection sweep...)
3. Need to have at least 2/3 models, one binary to identify song/nocall , then maybe frequency buckets by freq class, then birds

###### Version History, zoBirdFinder5A
* v1 - Moved from v7B of same, Intending to be new data creation, with B of same as Submit version



In [ ]:
## Kudos to the help from the community:
## some initial layout from:
# https://www.kaggle.com/muhakabartay/birdcall-eda-basemap-geo-3d-elevation-submission/


## Imports

In [ ]:
import numpy as np 
import pandas as pd 

import os

import IPython.display as ipd
pd.set_option('max_columns', 35)
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import pickle # serializing models  
# save model
# pickle.dump(model, open(filename, 'wb'))
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

import shutil
# shutil.copy2('/src/dir/file.ext', '/dst/dir/newname.ext') # complete target filename given
# shutil.copy2('/src/file.ext', '/dst/dir') # target filename is /dst/dir/file.ext

In [ ]:
# Imports


import matplotlib.image as mpimg
from matplotlib.offsetbox import AnnotationBbox, OffsetImage

# Map 1 library
import plotly.express as px

# Map 2 libraries
import descartes
import geopandas as gpd
from shapely.geometry import Point, Polygon

#install numba==0.48
# above is hack to not hit 0.5 removal of decorators

import numba
#==0.48
import numba.decorators

import sklearn

import warnings
warnings.filterwarnings('ignore')

# Audio / Threading Imports

In [ ]:
import librosa
import librosa.display
from joblib import Parallel, delayed, cpu_count
import time, math # for display type use ME


import pylab

import torchaudio
from scipy import signal

import gc
from path import Path

In [ ]:
import threading, queue

# q = queue.Queue()
bRUNNING = True

In [ ]:
bUseNoiseReduce = False
if bUseNoiseReduce == True:
## Imports for testing
    !pip install noisereduce

    import noisereduce as nr # noise reduction

In [ ]:
bSUBMITRUN = False
bPREDICTRUN = False ## Set to False for Data, True for Predict runs (automatically)
bFASTRUN = True
bRUNPARALLEL = True

nSAMPLERATE = 22050
bFILTER = True # butter filter
nMAXSONGITER = 5

# Multi-Threading Update





## Data Files

In [ ]:


model_filename = 'BFinder_model.sav'  # if first run, copy file to output folder (for path consistency..)
if os.path.isfile(model_filename):
    print("Model File already exists in output.")
else:
    shutil.copy2('/kaggle/input/zobirdsongdata3/BFinder_model.sav', 'BFinder_model.sav') # target filename is /dst/dir/file.ext
    print("Copying from saved model to output folder")
# Step on file in output if not saved

TEST = Path("../input/birdsong-recognition/test_audio").exists()
bSUBMITRUN = TEST
if TEST:
    DATA_DIR = str(Path("../input/birdsong-recognition"))
    print("SUBMIT PATH EXISTS")
else:
    # dataset created by @shonenkov, thanks!
    DATA_DIR = str(Path("../input/birdcall-check"))
    print("NON-SUBMIT - Using Alternative PATH")

print("DATA_DIR (Dynamic) = ",DATA_DIR)


TEST_FOLDER = DATA_DIR + '/test_audio/'
test = pd.read_csv(DATA_DIR + "/test.csv")

test.head()

In [ ]:
def onlyNAmerica(data, data_info=[]): # top 3 are N America, will filter out of data, based on data_info (in case different)
    if len(data_info)<1:
        data_info = data
    top_3 = list(data_info['country'].value_counts().head(3).reset_index()['index'])
    dInfo_top_3 = data_info[data_info['country'].isin(top_3)]
    data_top_3 = data[data['filename'].isin(dInfo_top_3['filename']) ]  #logical_not = data[~data
    return data_top_3

In [ ]:
TRAIN_DIR = str(Path("../input/birdsong-recognition"))
train = pd.read_csv(TRAIN_DIR + "/train.csv")
sub = pd.read_csv(TRAIN_DIR + "/sample_submission.csv")

# #######  train = onlyNAmerica(train)


## OUPUT FILES
train_part1 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-0_10.csv')
train_part2 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-10_15.csv')
train_part3 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-15_25.csv')
train_part4 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-25_40.csv')
train_part5 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-40_55.csv')
train_part6 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-55_73.csv')
train_part7 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-73_85.csv')
train_part8 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-85_95.csv')
train_part9 = pd.read_csv('/kaggle/input/zobirdsongdata2/Bproj_data0801-95_100.csv')
#../input/zobirdsongdata2/Bproj_data0801-0_10.csv
train_part1b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_0_4.csv')
train_part2b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_5_20.csv')
train_part3b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_20_35.csv')
train_part4b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_35_50.csv')
train_part5b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_50_65.csv')
train_part6b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_65_80.csv')
train_part7b = pd.read_csv('/kaggle/input/zobirdsongdata3/Bproj_data_80_95.csv')

print("Shape train: ", train.shape)
print("Shape  test: ", test.shape)
print("Shape   sub: ", sub.shape)

In [ ]:
test.head()

In [ ]:

# def addPathToTest(test, root_path=TEST_FOLDER):
#     test['path'] = root_path + test['audio_id'] + '.mp3'      
#     return test
    
# test_path = addPathToTest(test)

# test_path.head()
# urlSong = test_path.loc[0,'path']
# sound_clip = load_test_clip(urlSong,0)

# Create Q Rows Code
============================================================================



In [ ]:


bPREDICTRUN = True

def trainCreateQrow(filename, duration=None, bird=''):
    dfReturn = pd.DataFrame([['', '', 0, '']],\
                   columns=['site', 'row_id', 'seconds', 'audio_id'])
    
    if len(filename)>5:
        head, tail = os.path.split(filename)
        file = os.path.splitext(os.path.basename(filename))[0] # no extension 
    else:
        file = "NA"
        
    dfReturn['audio_id'] = file
    dfReturn['row_id'] = file
    dfReturn['seconds'] = None
    dfReturn['site'] = 'site_3' ## anything mostly
    print(filename)
    return createQrow(dfReturn, 0, fSecFile=None, root_path=head+'/' )



def createQrow(test, nIndex, fSecFile=None, root_path=TEST_FOLDER):
    bIterate = True
    dfReturn = pd.DataFrame()
    rowTest = test.iloc[int(nIndex)]
    rowTest['path'] = root_path + rowTest['audio_id'] + '.mp3'

#     fSecFile=11 ### HACK for testing
    if (rowTest['site'] == 'site_1') or (rowTest['site'] == 'site_2'):
        rowTest['nIter'] = (rowTest['seconds'] /5)-1
        dfReturn = dfReturn.append(rowTest, ignore_index=True)
#     elif fSecFile == None:
#         ## ERROR
#         print("ERROR, Site 3 has not Duraton passed in.")
#         rowTest['nIter'] = -1
    else:
        if fSecFile == None:
            print("Loading....")
            fSecFile = librosa.get_duration(filename=rowTest['path'])
            print("done, fSecFile = ", fSecFile)
#         print("Site 3 has not Duraton passed in.")
        secIteration =  5.0
        nTotIter = math.ceil(fSecFile / secIteration)
        if bPREDICTRUN == False:
            if bIterate == True:
                secLastIter = fSecFile % secIteration  
                if secLastIter < 1.5 and fSecFile > secIteration:
                    nTotIter = nTotIter-1 # Last iter too short, let's skip it  ### Keep if Submission file??
            else:
                nTotIter = 1
            if nTotIter > nMAXSONGITER and bPREDICTRUN == False:
                nTotIter = nMAXSONGITER # limit to this number of segments       
        secEnd = secIteration       
        for nIter in range(nTotIter):
            if fSecFile <= secEnd:
                secEnd = fSecFile  
            rowTest['nIter'] = nIter
            rowTest['seconds'] = (nIter+1)*5.0 ## endTime
            secStart = secEnd
            secEnd = secEnd + secIteration
            dfReturn = dfReturn.append(rowTest, ignore_index=True)
            
    nRetSize = len(dfReturn)
    return dfReturn, nRetSize




# dfTest = pd.DataFrame()
# for rowID in range(len(test)):
#     dfTest = dfTest.append(createQrow(test,rowID), ignore_index=True)

    
# dfTest[-30:]   



In [ ]:
sampleBird_file1 = '/kaggle/input/birdsong-recognition/train_audio/astfly/XC122918.mp3'



In [ ]:

# trainCreateQrow('07ab324c602e4afab65ddbcc746c31b5.mp3', None) #
# trainCreateQrow('41e6fe6504a34bf6846938ba78d13df1.mp3', None) #sampleBird_file1

In [ ]:
### N. America Filtered data
trainNA = onlyNAmerica(train)
print("Filter our migration data:  All, N.America = ",len(train), len(trainNA))
dbNAEbirdCodes = trainNA['ebird_code'].unique()
dbNAEbirdCodes=np.append(dbNAEbirdCodes,'nocall')
dbNAEbirdCodes=np.append(dbNAEbirdCodes,'unknown')
nEbirdCodesNA = len(dbNAEbirdCodes) # same birds, migration data is skipped however

In [ ]:

dbEbirdCodes = train['ebird_code'].unique()
dbEbirdCodes=np.append(dbEbirdCodes,'nocall')
dbEbirdCodes=np.append(dbEbirdCodes,'unknown')
nEbirdCodes = len(dbEbirdCodes)
enumECodes = dict(zip(dbEbirdCodes,range(nEbirdCodes)))  # enumECodes['wooscj2'] -> =255
enumECodesRev = dict(zip(range(nEbirdCodes),dbEbirdCodes))

print("ECodes = ", nEbirdCodes,"total, ", dict(list(enumECodes.items())[-20:]))
print("  ##  ")
print("enumECodesRev = ", nEbirdCodes,"total, ", dict(list(enumECodesRev.items())[-20:]))

In [ ]:
def countTrainBirds(data):
    print("Shape train: ", data.shape)
    dbTrainBirds = data['ecode'].unique()
    print("Training set has ", len(dbTrainBirds), " unique birds in it.")

    dbMissingEBirds = []
    dbMissingBirds = []
    for bird in range(nEbirdCodes):
        if bird not in dbTrainBirds:
            dbMissingEBirds = np.append(dbMissingEBirds, bird)
            dbMissingBirds = np.append(dbMissingBirds, enumECodesRev[bird])
    print("Training set Missing ecodes =  ",dbMissingEBirds)
    print("Training set Missing birds =  ",dbMissingBirds)
    return len(dbMissingEBirds)

In [ ]:

bird_columns = ['nSongID', 'ecode','file', 'nUpRows', 'nDownRows','10',\
                'nCrossings','bArea', 'sNotes', 'sLength', 'sRange', 'sTrendline', 'sPitch', 'sSpacing', 'sDutyCycle',\
               ]
bird_dbRow = [0,'None','None',0, 0,0,\
             0, 'None', 0, 0.0, 0.0, 0.0, 0.0,0.0, 0.0, \
             ]

birdRow = pd.Series(data=bird_dbRow, index=bird_columns)
birdDb = pd.DataFrame(columns = bird_columns)
newBirdRow = birdRow.copy()


In [ ]:
## USED
bFinder_columns = ['nSongID', 'ecode','file_nIt', 'primFr', 'avgFr','primSl','dCycle', 'nSegm','fFundFreq','nFFmult', \
                 'f1', 'f2','f3','f4', 'f5','f6','f7','f8', 'f9','f10',\
                 'fp1', 'fp2','fp3','fp4', 'fp5','fp6','fp7','fp8', 'fp9','fp10',\
               ]
bFinder_row = [0,0,'None',0.0, 0.0,0.0, 0.0,0.0, 0.0,0.0, \
             0.0,0.0, 0.0,0.0, 0.0,0.0, 0.0,0.0, 0.0,0.0, \
             0.0,0.0, 0.0,0.0, 0.0,0.0, 0.0,0.0, 0.0,0.0,\
             ]

bFinderRow = pd.Series(data=bFinder_row, index=bFinder_columns)
bFinderDb = pd.DataFrame(columns = bFinder_columns)
newBFinderRow = bFinderRow.copy()

# Setting up Que's

q = queue.Queue()

In [ ]:
# Que for songs waiting processing, labelled as done afterwards
queToProcess_columns = ['nIndex', 'Species', 'ecode','file', 'nBirdRefId', 'nStatus','startSec', 'duration' ]
queToProcess_row = [0, 'None', 'None','None', 0, 0, 0.0, 0.0 ]
queToProcess = pd.DataFrame(columns = queToProcess_columns)

qToProcess_columns = ['nIndex','file', 'fileRefId', 'nStatus' ,'startSec', 'duration', 'nIter', 'nLoc']
qToProcess_row = [0, 'None', 'None', 0 , 0.0, 0.0, 0,0 ]
newToProcessRow = pd.Series(data=qToProcess_row, index=qToProcess_columns)
qToProcess = queue.Queue(maxsize=6) # maxsize unlimited.. 3 worked


## Main Starting functions

In [ ]:
def startThreads():
            
    # turn-on the worker thread
    threading.Thread(target=workerFileQ, daemon=True).start()

# if __name__ == '__main__':
#     main()
    return


def initRun():
    initDb()
    nPipeRuns = 0
    nFFTGraphs = 0
    return

def initDb():
    global birdDb
    global bFinderDb
    global queToProcess
#     global queSongs
    global queSongInfo
    global bRUNNING
    birdDb.drop(birdDb.index, inplace=True) # flush db of rows
    bFinderDb.drop(birdDb.index, inplace=True) # flush db of rows
    queToProcess.drop(queToProcess.index, inplace=True) # flush db of rows
#     queSongs.drop(queSongs.index, inplace=True) # flush db of rows
#     queSongInfo.drop(queSongInfo.index, inplace=True) # flush db of rows
    bRUNNING = False
    time.sleep(4) # Allow child threads to stop
    bRUNNING = False
    # Start threads...
    startThreads()
    return


## Main support functions

In [ ]:


def newRow():
    global newBirdRow
    newBirdRow = birdRow.copy()
    return
def newBRow():
    global newBFinderRow
    newBFinderRow = bFinderRow.copy()
    return

def addRowValue(paramName = "None", paramValue = "0.0"):
    global newBirdRow
    newBirdRow[paramName] = paramValue
    return
def addBRowValue(paramName = "None", paramValue = "0.0"):
    global newBFinderRow
    newBFinderRow[paramName] = paramValue
    return

def appendRow():  
    global birdDb
    global newBirdRow 
    birdDb = birdDb.append(newBirdRow, ignore_index=True)
    nIndex =  len(birdDb.index)-1
    return nIndex  # return index of new row
def appendBRow():  
    global bFinderDb
    global newBFinderRow 
    bFinderDb = bFinderDb.append(newBFinderRow, ignore_index=True)
    nIndex =  len(bFinderDb.index)-1
    return nIndex  # return index of new row


def getBirdRow( nIndex = -1, lSpecies = "None", nSpecies = -1):
    global birdDb
    global newBirdRow
    if nIndex >= 0 and nIndex in birdDb.index: # if valid
        lSpecies = birdDb.at[nIndex,'Species']
        nSpecies = birdDb.at[nIndex,'nSpecies']
    else:
        if nSpecies < 0:
            nSpecies = 0
        dbSpecList = birdDb.loc[ birdDb['Species'] == lSpecies].index
        print("dbSpecList = ", dbSpecList)
        if len(dbSpecList) > 1:           
            nIndex = dbSpecList[nSpecies]
        else:
            nIndex = dbSpecList.index[0]        
#                 
    newBirdRow = birdDb.iloc[nIndex,:].copy()
#     print("nIndex, Species, nSpec = ", nIndex , lSpecies , nSpecies )
    return nIndex
def getBFinderRow( nIndex = -1, ecode = "None", nSongID = -1):
    global bFinderDb
    global newBFinderRow 
    if nIndex >= 0 and nIndex in bFinderDb.index: # if valid
        ecode = bFinderDb.at[nIndex,'ecode']
        nSongID = bFinderDb.at[nIndex,'nSongID']
    else:
        if nSongID < 0:
            nSongID = 0
        dbSpecList = bFinderDb.loc[ bFinderDb['ecode'] == ecode].index
        if len(dbSpecList) > 1:           
            nIndex = dbSpecList[nSongID]
        else:
            nIndex = dbSpecList.index[0]         
    newBFinderRow = bFinderDb.iloc[nIndex,:].copy()
#     print("nIndex, Species, nSpec = ", nIndex , ecode , nSongID )
    return nIndex

def updateRow( nIndex = -1):
    global birdDb
    global newBirdRow
    if nIndex < 0:
        return -1
    birdDb.iloc[nIndex,:] = newBirdRow
    return 0
def updateBRow( nIndex = -1):
    global bFinderDb
    global newBFinderRow 
    if nIndex < 0:
        return -1
    bFinderDb.iloc[nIndex,:] = newBFinderRow
    return 0    
    

def predBird():
    return

def runPipeline():
    return

def trainBirdModel():
    return

def enableAccell():
    return

def splitWork():
    return

def outputResults():
    return

In [ ]:


def saveData(data, strFileName = "proj_data"):
    filename = strFileName +".csv"
    data.to_csv(filename, index=False)
    return

def saveSubmission(sub):
    saveData(sub, "submission")
#     sub.to_csv('submission.csv', index=False)
    return

# Stat Creation

In [ ]:
# Create Full Path so we can access data more easily
# ROOT_INPUTPATH = 'C:\\Users\\meckd\\PycharmProjects\\birdsong1\\data\\birdsong-recognition\\'
ROOT_INPUTPATH = '/kaggle/input/birdsong-recognition/'

base_dir = ROOT_INPUTPATH + 'train_audio/'
train['full_path'] = base_dir + train['ebird_code'] + '/' + train['filename']


In [ ]:
def loadFile(file, sr=nSAMPLERATE):  # Add limit to length or file, or start/end..
    x, x_sr = torchaudio.load(file)
    x = x.numpy()

    if x.ndim > 1:
        x = np.mean(x, axis=0)
    bReduceNoise = False
    if bReduceNoise:
    # Noise reduction
        noisy_part = x[0:25000]  ### Warning need to id noisy part to pull
        x = nr.reduce_noise(audio_clip=x, #Reduced noise here
                        noise_clip=noisy_part, verbose=False)

    if sr > 0 :
        x = signal.resample(x, int(x.size*float(sr)/x_sr))
        x_sr = sr
        
    return x, x_sr

In [ ]:
def freqCutOff(f,m): # trim low and too high freq data 
#     fCut = np.logical_and((30 < f), (f<10000)) # return array of t/f not quite right multiply parters
    fCutLow = (30 < f) # return array of t/f
    fCutHigh =  (f<10000) # return array of t/f  10k is about half of current 22k sampling
    f = (f* fCutLow) * fCutHigh
    m = (m* fCutLow) * fCutHigh
    return f,m

In [ ]:
from scipy import fft, arange
def frequency_sepectrum(x, sf=nSAMPLERATE): 
    # https://stackoverflow.com/questions/53308674/audio-frequencies-in-python/53309191#53309191
    
    """
    Derive frequency spectrum of a signal from time domain
    :param x: signal in the time domain
    :param sf: sampling frequency
    :returns frequencies and their content distribution
    :NOTE: Hearing is generally 28Hz-20kHz, but freq cutoff with 22k sampling is 11kHz
    """
    if len(x)<100:
        return [],[]
    
    x = x - np.average(x)  # zero-centering

    n = len(x)
#     print(n)
    k = arange(n)
    tarr = n / float(sf)
    frqarr = k / float(tarr)  # two sides frequency range

    frqarr = frqarr[range(n // 2)]  # one side frequency range

    x = fft(x) / n  # fft computing and normalization
    x = x[range(n // 2)]
    # Added by me to normalize:
    x= abs(x)
    x = x / (np.max(x)/2) # similar to 0-2 range

    return freqCutOff(frqarr,x)  # return freq's and magnitudes after fCutting to limit range

In [ ]:
from scipy.signal import butter, lfilter, freqz
# https://stackoverflow.com/questions/25191620/creating-lowpass-filter-in-scipy-understanding-methods-and-units


## This code replaces above 4 functions
def butter_Xpass_filter(data, cutoff, fs, order=5, bHighPass=True): # cutoff is in Hz, fs is sampling rate, order is filter orderQQ  
    nyq = 0.5 * fs
    normal_cutoff = cutoff / nyq
    if bHighPass==True:
        btype='high'
    else: # Low Pass Filter instead, (blocks high)
        btype='low'
    b, a = butter(order, normal_cutoff, btype=btype, analog=False)
    y = lfilter(b, a, data)
    return y

In [ ]:
nFFTGraphs = 0
def graphFFT(freqs, mags): 
#     fig, ax = plt.subplots(1, figsize = (16, 9))
#     fig.suptitle('Sound Waves', fontsize=16)
    global nFFTGraphs
    nFFTGraphs +=1
    if (nFFTGraphs)>8:
        return # limit graphing
    fig2 = plt.figure(1, figsize = (12, 4))
    plt.plot(freqs,mags)
    plt.xlabel('frequency')
    plt.ylabel("Magnitude") 
    plt.show()
    return

In [ ]:
def getMaxP(data):   # get max Power, indexP, and FreqP in this order...
    mFreq, mPower, nMaxF = getMaxFrqPwr(data)
    return mPower, nMaxF, mFreq        #fMaxP, nMaxPId, fMaxPFreq ## order change???

In [ ]:
def getMaxFrqPwr(data):  ## right now max power is always 2 due to scaling to 2
    freqs, mags = frequency_sepectrum(data)
    nMaxF =  (mags**2).argmax() # mags.argmax() #
    mFreq = round(freqs[nMaxF],2)      #((freqs[nMaxF])*100//1)/100
    
    avgMag = np.mean(mags)*2 # trying to get more range
    mPower = abs(50*(2-avgMag) )    ## mags[nMaxF] Always 2  # Diff mean to top times 100/2.0, so ranged to 100 max
    
    graphFFT(freqs, mags) # will graph until stopped by count
    return mFreq, mPower, nMaxF

In [ ]:
def calcFreqStats(data,saveData = True, fMinTh=0.0, showGraph = False):
    nLength = len(data)
    if nLength<100:
        return 0,0
    
    secData = nLength / nSAMPLERATE  
    mFreq, mPower, nMaxF = getMaxFrqPwr(data)   
#     print("sec long, maxP(of 100) = ", secData,  mPower)   
    return mFreq, mPower

In [ ]:
# https://www.kaggle.com/seshganesh/dataprocessing

def calcFreqStats1(data, fMinTh=0.0, saveData = True, showGraph = False):
    global lastData
    if saveData == True:
        lastData = data
#     else:
#         data = lastData # using saved data
    lastData = data # making sure data gets here
    if len(data)<100:
        return 0,0
        
    fMinTh=0.0 # testing   
    ##########################################################################
    freqs, mags = frequency_sepectrum(data)

    nMaxF = mags.argmax() # (mags**2).argmax() 
    mFreq = ((freqs[nMaxF])*100//1)/100
    mPower = mags[nMaxF]
    
    return mFreq, mPower

In [ ]:
def addBFinderRow(nSongID=0, ecode='', file='', dbPrimData=[] , dbFreqs=[], dbFingerprint=[] ):
    global bFinderDb
#     global bFinder_columns

    lenDBPRs = len(dbPrimData) 
    lenDBFs = len(dbFreqs)
    lenDBFPs = len(dbFingerprint)
    if lenDBPRs < 7:# Fill rows if needed
        for row in range (7-lenDBPRs):
            dbPrimData.append( 0.0 )
    if lenDBFs < 10:        
        for row in range (10-lenDBFs):
            dbFreqs.append( 0.0 )
    if lenDBFPs < 10:
        for row in range (10-lenDBFPs):
            dbFingerprint.append( 0.0 )

    del dbFreqs[10:] # was behind and else...
    del dbFingerprint[10:]
    
    bNewBFind_row = [nSongID, ecode,file]  # 0-2
    bNewBFind_row.extend(dbPrimData)    # 3-9
    bNewBFind_row.extend(dbFreqs)       # 10-19
    bNewBFind_row.extend(dbFingerprint) # 20-29
#     print("bNewBFind_row = ", bNewBFind_row)
    thisBFindRow = pd.Series(data=bNewBFind_row, index=bFinder_columns)
    
    bFinderDb = bFinderDb.append(thisBFindRow, ignore_index=True)
#     print("freqDb = ", bFinderDb)
    
    return

In [ ]:
def peakFingerprint(freqs, delays, mags):  # Note freqs is condensed freq, and delay is the delay before each Freq point
    fp = []
    fTotFSlope=0.0
    fTotMSlope=0.0
    fTotDelays = 0.0
    frqN = len(freqs)
    if frqN<1:
        return fp
    elif frqN == 1:
        fp = [freqs[0],1,0] # Peak, slope, magn to next
        
    fplus1,dplus1,mplus1 = 0.0, 0.0, 0.0
    for nFrq in range(frqN):
        if frqN > nFrq+2:
            fplus1 = freqs[nFrq+1]
            dplus1 = delays[nFrq+1]
            mplus1 = mags[nFrq+1]
            
            m = (fplus1 - freqs[nFrq])/(22000/(dplus1+1)) # added * (steps/22000) as run
            fSlope = math.degrees(math.atan(m ))#//1 # angle from x axis
            mMag = (mplus1 - mags[nFrq])/(22000/(mplus1+1)) # added * (steps/22000) as run
            fMagSlope = math.degrees(math.atan(mMag ))#//1 # angle from x axis
            
            fTotFSlope += fSlope
            fTotMSlope += fMagSlope
            fTotDelays += delays[nFrq]

    fp.append(fTotFSlope)
    fp.append(fTotMSlope)
    fp.append(fTotDelays)

    return fp

In [ ]:
def getCrossingCount2(data): 
    nPoints = len(data)
    maxPoint = abs(data).max()
#      sum(zero_crossings)  returns boolean array 
    zero_crossings = sum(librosa.zero_crossings(data, pad=False))    #(x[nStart:nEnd], pad=False) # pad is if x[0] is counted as cross
#     print("zero_crossings, nPoints = ", zero_crossings, nPoints)
    fHz = (zero_crossings * nSAMPLERATE / (2* nPoints) ) # (2 * (len(chunk))))
    return fHz, maxPoint

In [ ]:
# import numpy as np
# from numpy import logical_and, average, diff

def freq_from_crossings(sig, fs=nSAMPLERATE):
    """Estimate frequency by counting zero crossings
      
    Doesn't work if there are multiple zero crossings per cycle.
    
    """
    indices = np.nonzero(np.diff(sig > 0))[0]
#     indices = find(logical_and(sig[1:] >= 0, sig[:-1] < 0))
    
    # Linear interpolation to find truer zero crossings
    crossings = [i - sig[i] / (sig[i+1] - sig[i]) for i in indices]
    return fs / np.average(np.diff(crossings))

In [ ]:
# Going to try and standardize on 5 sec times, with overlap  NOTE 1/2 OVERLAP NOW ######
def calcFreqSweep(nDiv=100, data=[], chirp_data=[], bUseFFT=True, fMinTh=0.0): # divmod or // for int division
    global lastData
    freq = []
    mag = []
    nPoints = len(data)
    if nPoints>100:
        lastData = data
    else:
        nPoints = len(lastData)
    
    frMax,magMax = calcFreqStats(lastData, saveData=False)
    magMin = fMinTh
    
    if nPoints > (nSAMPLERATE*5):
        nPoints = (nSAMPLERATE*5) # 5 sec.
        
    nOverlap = 1     # 2 = 1/2 overlap, 1 for no overlap, etc
    nStepSize = (nPoints // nDiv)*nOverlap
#     print("nStepSize = ", nStepSize)
    start = 0
    nTotSteps = nDiv-nOverlap+1
    for step in range(nTotSteps):
        stop = start + nStepSize
        if step == nDiv-1:
            stop = nPoints
            
        #check if data over 0
        m = np.max(chirp_data[start:stop])
        if m >1:
            
            if bUseFFT == True:
                f,_ = calcFreqStats(lastData[start:stop], fMinTh=fMinTh, saveData=True)
            else:
#                 f0, fSpacing = getCrossingCount(lastData[start:stop], fMinTh=0.0) 
                f, m2 = getCrossingCount2(lastData[start:stop]) 
#                 f = freq_from_crossings(lastData[start:stop]) 
#                 if m != m2:
#                 print("Not Equal m, m2 = ",  m, m2)
#                 print("Not Equal f0, f, f3 = ",  f0, ", ", f,", ", f3)
#                 m=0
        else:
            f = 0.0
            
#         print("calc Freq start, stop", start, stop, f)
        if (f <= 23.0) or (m2 < 0.25):## m2 added need to test
            f = 0.0
            m = 0.0
        start = start + (nStepSize//nOverlap)
        freq.append(f)
        mag.append(m)
#     if bUseFFT == True:
#         print("FFT - freq found = ", freq) # output all Freq, delays
#     else:
#         print("Spacing - freq found = ", freq) # output all Freq, delays
    return freqCompress(freq, mag)  #################### LOSING Magnitude 1-100 measurement here!!!!

###################


def freqCompress(freq, mag):
    delayDb = []
    freqDb = []
    magnDb = []
    mag = mag
    nDelayCount=0
    nCount = 0
    nFeqCount = 0
    for (f,m) in zip(freq,mag):
        if f>20:
#             if nDelayCount>-1 or nCount==0:
            delayDb.append(nDelayCount)
            freqDb.append(f)
            magnDb.append(m)
            nDelayCount=0
            nFeqCount+=1
        else:
            nDelayCount+=1
        
        
        nCount +=1
        if nFeqCount>=100:
            break
#     print("Freq, Delay = ", freqDb, delayDb) 
    if len(freqDb) < 3: ## hack for later indexing...
        threeZeros = [0.0, 0.0, 0.0]
        freqDb.extend(threeZeros)
        magnDb.extend(threeZeros)
        delayDb.extend(threeZeros)
        
    
    fp = peakFingerprint(freqDb, delayDb, magnDb)
    return freqDb, magnDb, fp
##################

def freqCompress2(freq, mag): ## unused, experimental now
    delayDb = []
    freqDb = []
    magnDb = []
    mag = mag
    nDelayCount=0
    nCount = 0
    nFeqCount = 0
    lastF = -1
    lastM = -1
    magMax = max(mag)
      
    for (f,m) in zip(freq,mag):
        if lastF < 0:
            lastF = f
            lastM = m
        if (f>20) and (f>(magMax/4) ):
            freqDb.append(f) # was lastF-f
            magnDb.append(lastM-m)
                       
                       
            riseM = (sum(magnDb))/(22000/(len(delayDb)*10 +1)) # added * (steps/22000) as run
            fSlope = math.degrees(math.atan(riseM )) # angle from x axis  was //1
            delayDb.append(fSlope)     ## This jacks up fingerprint      
            nFeqCount+=1   
            lastF = f
            lastM = m
#         else:                       
        
        nCount +=1
        if nFeqCount>=100:
            break
            
#     print("Freq, Delay = ", freqDb, delayDb) 
    if len(freqDb) < 3: ## hack for later indexing...
        threeZeros = [0.0, 0.0, 0.0]
        freqDb.extend(threeZeros)
        magnDb.extend(threeZeros)
        delayDb.extend(threeZeros)
        
    
    fp = peakFingerprint(freqDb, delayDb, magnDb)
    return freqDb, magnDb, fp #delayDb # fp ### HACKKKK  # F, D, Fp expected


## New fingerprint
    def fp(secWidth, secMiddle, totPts, midPts)
*     Setup buckets for beginning, middle, end of peaks, with middle at top of main peak
*     Buckets initially 20, 20 middle, 20 end
*     Measure peak power and crossings (5 crossings is 2 waves), calc partial crossings too, with leftover percentage
*     Be able to graph P, m for fp
*     initial secWidth = 0.2, secMiddle = 0.04, so .005 sec bucket, .0025 sec/buck, .005
 
>     This will be too wide for slow  and too narrow for high, but should be able to identify still...
>     Potentially could vary based on avg Freq (by larger buckets...)

In [ ]:
nCCRuns = 0
def getCrossingCount(data,fMinTh=25): 
    maxPoint = data.max()
    nMaxPtIndex = data.argmax()
    minThreshold = fMinTh 
    nPoints = len(data)
    global nCCRuns
    nCCRuns +=1
    nStart = nMaxPtIndex -40
    nEnd = nMaxPtIndex +40
    if nStart < 0:
        nStart = 0
    if nEnd >= nPoints-1:
        nEnd = nPoints-1
        
#     print("nSt, nEnd = ", nStart, nEnd)
        
    subData = data[nStart:nEnd]
    nCrossings = 0
    
    bOver = subData > minThreshold
    bLastValue = bOver[0]
    nFirstID = -1
    nLastID = -1
    nIndex = 0
    nBefore = 0
    for bValue in bOver:
        if bLastValue != bValue:
            nCrossings += 1
        bLastValue = bValue
        if nFirstID < 0 and bValue >0:
            nFirstID = nIndex # first up peak
        if bValue > 0:
            nLastID = nIndex # last up peak end
        nIndex += 1
        
    if nCrossings >0:
        fSpacing = (nEnd-nStart)/((nCrossings)/2)  # /2 to account for 4 crossings / lambda
    else:
        fSpacing = 1
    
    if fSpacing >0:
        fHz = nSAMPLERATE/fSpacing
    else:
        fHz = 1
    
    fHz= (fHz*100//1)/100
    fSpacing= (fSpacing*100//1)/100
#     if nCCRuns < 5:
#         print("Crossing Count peak data: ", subData)
#         print("Crossings, fSpacing, st, end = ",nCrossings ,fSpacing, nFirstID, nLastID)
    
    return fHz, fSpacing

In [ ]:
def getPeakSpacing(data,fMinTh=0.0): 
    
    maxPoint = data.max()
    nMaxPtIndex = data.argmax()
    minThreshold = fMinTh #maxPoint*(minThPercent/100)
    fSpacing = 0.0
    nPoints = len(data)
    
    nStart = nMaxPtIndex -200
    nEnd = nMaxPtIndex +200
    if nStart < 0:
        nStart = 0
    if nEnd > nPoints:
        nEnd = nPoints
        
    subData = data[nStart:nEnd]
#     print("subData = ", subData)
#     print("minThreshold = ", minThreshold)   
    
    nUpCrossings, nDownCrossings = 0,0
    nUpRows,nDownRows = 0,0
    bUp,bDown,bUpCrossings,bDownCrossings = False,False,False,False

    dbIndex = []
    dbPower = []
    dbSlope = []
       
    index = 0
    nPeaks = 0
    lastSlope = None
    lastId = -1
    lastPow = None
    lastIdSlope = -1
    for row in subData:
        index +=1
        if row > minThreshold:
            if bUp == False:
                bUp = True
                bUpCrossing = True
                nUpCrossings +=1
#                 print("Index, Data = ", index, row)
                dbIndex.append(index)
                dbPower.append(row)
                fSlope = row-lastPow
                lastPow = row
#             dbDiffPow.append( fSlope)
                if nPeaks >0 and (dbPower[nPeaks] - dbPower[nPeaks-1])>1:
                    dbSlope.append(dbIndex[nPeaks] - dbIndex[nPeaks-1])

            else:
                bUpCrossing = False
            bDown = False
            bDownCrossing = False
            nUpRows +=1

        if row < minThreshold:
            if bDown == False and bUp == True:
                bDown = True
                bDownCrossing = True
                nDownCrossings +=1
            else:
                bDownCrossing = False
            bUp = False
            bUpCrossing = False
            nDownRows +=1
            
#     print("subData = ", subData)
    nSubPoints = len(subData)
    
    fSpacing = nSubPoints/len(subData)
   

#     print("Number Up Rows, Number Down, nCrossings = ", nUpRows, " , ", nDownRows," , ",nDownCrossings)

    fDutyCycle = nUpRows / nDownRows

    return fSpacing

# Calc Stats HERE --------------------

In [ ]:
def findFundFreq(fMaxPFreq,fFreqMean=0 ):
    
    ### Adding here now, will pull to function # range 1800-3600
    nTypRange = -1
    if fFreqMean >1650 and fFreqMean <1950:
        nTypRange = 1
    if fFreqMean >3300 and fFreqMean <3900:
        nTypRange = 2
            
    if fMaxPFreq <= 225.0:  
        fFundFreq = fMaxPFreq * 16.0
        nFFmult = -4  # multiplier factor relative to FF range
    elif fMaxPFreq <= 455.0:  
        fFundFreq = fMaxPFreq * 8.0
        nFFmult = -3
    elif fMaxPFreq <= 900.0:  
        fFundFreq = fMaxPFreq * 4.0
        nFFmult = -2
    elif fMaxPFreq > 1650 and fMaxPFreq <= 1800 and nTypRange == 1: # special case
            fFundFreq = fMaxPFreq * 1.0
            nFFmult = 0    
    elif fMaxPFreq <= 1800.0: # 900 - 1799
        fFundFreq = fMaxPFreq * 2.0
        nFFmult = -1       
    elif fMaxPFreq <= 3600.0:  # 1800 - 3600
        fFundFreq = fMaxPFreq * 1.0
        nFFmult = 0
    elif fMaxPFreq > 3300 and fMaxPFreq < 3900 and nTypRange == 2: # special case
            fFundFreq = fMaxPFreq * 1.0
            nFFmult = 0  
    elif fMaxPFreq <= 7200.0:  # 3600 ++
        fFundFreq = fMaxPFreq * 0.5
        nFFmult = 1
    else:  
        fFundFreq = fMaxPFreq * 0.5  ## Most likely better not adding another level
        nFFmult = 1    
    print("ff = ", fFundFreq, nFFmult)
    return fFundFreq, nFFmult
           

In [ ]:
#  calcStats(audio_bird, filename=file, ecode = ecode, nIndex=objSong.nIndex)
##  ### This is fed 5 sec sections of data, file is only for reference
def calcStats(data, audio_chirp, minThPercent=25, filename="", ecode = "", nIndex=0 , fDutyCycle = "", nIter=1):
    global bPREDICTRUN

    if len(filename)>5:
        file = os.path.basename(filename) # this is only for db name, file already read
    else:
        file = "NA"
#     fDutyCycle = params
    
    nLength = len(data)
    if nLength > (nSAMPLERATE*1):
        nEndLength = nLength - int(nSAMPLERATE*0.5) # 5 sec.
    else:
        nEndLength = nLength
        
    nMaxAudioP = audio_chirp[:nEndLength].argmax()

    # Now set to 1/2 sec plus and minus from peak
    if nMaxAudioP > int(nSAMPLERATE*0.5):
        nMidStart = nMaxAudioP -int(nSAMPLERATE*0.5) # set back by 1/2 sec
    else:
        nMidStart = 0
        
    nMidEnd = nMidStart+int(nSAMPLERATE*0.5)
    
    fMaxP, nMaxP, fMaxPFreq = getMaxP(data[nMidStart:nMidEnd])
    
    dbF, dbD, dbFp = calcFreqSweep(40, data=data[nMidStart:nMidEnd], chirp_data=audio_chirp[nMidStart:nMidEnd], bUseFFT=False, fMinTh=0.3*fMaxP) # this is file based
    #######################################
    # dbPrimData = [ 'primFr', 'avgFr','primSl','dCycle', 'nSegm','fFundFreq','nFFmult', ]\
    fFreqMean = 0.0
    nFrq = 0
    for item in dbF:
        if item>30 and item < 10000:
            fFreqMean += item
            nFrq += 1
    if nFrq>0:
        fFreqMean = fFreqMean/nFrq  # was np.nanmean(dbF)
#         print("mean, n  = ", fFreqMean, nFrq)
    percDiff =  (abs(fMaxPFreq-fFreqMean)/fFreqMean)  ## could if 1.0 then /2, (.5 to x2) etc....
    
    if (fFreqMean>800):  ### Testing catching second harmonics
        if fMaxPFreq > fFreqMean and percDiff > 0.95 and percDiff < 1.05 :
            fMaxPFreq = fMaxPFreq/2 ## suposing 2nd harmonic
            print("fMaxPFreq = fMaxPFreq/2 ## suposing reverse 2nd harmonic = ",fMaxPFreq," , ", fFreqMean )
        if fMaxPFreq < fFreqMean and percDiff > 0.46 and percDiff < 0.54 :
            fMaxPFreq = fMaxPFreq*2 ## suposing reverse 2nd harmonic
            print("fMaxPFreq = fMaxPFreq*2 ## suposing reverse 2nd harmonic = ",fMaxPFreq," , ", fFreqMean )
            
    fFundFreq,nFFmult = findFundFreq(fMaxPFreq,fFreqMean )
    
#     ### Adding here now, will pull to function # range 1800-3600
#     nTypRange = -1
#     if fFreqMean >1650 and fFreqMean <1950:
#         nTypRange = 1
#     if fFreqMean >3300 and fFreqMean <3900:
#         nTypRange = 2
            
#     if fMaxPFreq <= 225.0:  
#         fFundFreq = fMaxPFreq * 16.0
#         nFFmult = -4  # multiplier factor relative to FF range
#     elif fMaxPFreq <= 455.0:  
#         fFundFreq = fMaxPFreq * 8.0
#         nFFmult = -3
#     elif fMaxPFreq <= 900.0:  
#         fFundFreq = fMaxPFreq * 4.0
#         nFFmult = -2
#     elif fMaxPFreq > 1650 and fMaxPFreq <= 1800 and nTypRange == 1: # special case
#             fFundFreq = fMaxPFreq * 1.0
#             nFFmult = 0    
#     elif fMaxPFreq <= 1800.0: # 900 - 1799
#         fFundFreq = fMaxPFreq * 2.0
#         nFFmult = -1       
#     elif fMaxPFreq <= 3600.0:  # 1800 - 3600
#         fFundFreq = fMaxPFreq * 1.0
#         nFFmult = 0
#     elif fMaxPFreq > 3300 and fMaxPFreq < 3900 and nTypRange == 2: # special case
#             fFundFreq = fMaxPFreq * 1.0
#             nFFmult = 0  
#     elif fMaxPFreq <= 7200.0:  # 3600 ++
#         fFundFreq = fMaxPFreq * 0.5
#         nFFmult = 1
#     else:  
#         fFundFreq = fMaxPFreq * 0.5  ## Most likely better not adding another level
#         nFFmult = 1    
            
    dbPrimData = [fMaxPFreq, fFreqMean, fMaxP,fDutyCycle, nIter, fFundFreq, nFFmult] # end rough slope

    if fMaxP >= 70.0 and fMaxPFreq >= 100.0 and fFreqMean >= 100.0:  ### DETERMINE IF ADDED TO DB !!!!!
#     if (bPREDICTRUN == True) or (abs(fMaxPFreq-fFreqMean)/fFreqMean) < 0.4 and fMaxP >= 90.0 :  ### DETERMINE IF ADDED TO DB !!!!!
        nECode = enumECodes[ecode] 
        fCode_Fn = ecode + '_' + os.path.splitext(file)[0]
        addBFinderRow(nSongID=nIndex, ecode=nECode, file=fCode_Fn, dbPrimData=dbPrimData , dbFreqs=dbF, dbFingerprint=dbFp )
    elif (bPREDICTRUN == True):
        nECode = enumECodes[ecode] 
        fCode_Fn = ecode + '_' + os.path.splitext(file)[0]
        dbPrimData = [0.0, 0.0, 0.0, 0.0, nIter, 0.0, 0] # force to "nocall" hopefully...
        addBFinderRow(nSongID=nIndex, ecode=nECode, file=fCode_Fn, dbPrimData=dbPrimData , dbFreqs=dbF, dbFingerprint=dbFp )
    elif fMaxP < 60.0 or fMaxPFreq < 100.0 or fFreqMean < 100.0:
        nECode = enumECodes["nocall"] # set as nocall
        fCode_Fn = ecode + '_' + os.path.splitext(file)[0]
        print("------- NO CALL ----- = Freq, avFre, fMaxP" ,fMaxPFreq,", ", fFreqMean," - ", fMaxP )
        addBFinderRow(nSongID=nIndex, ecode=nECode, file=fCode_Fn, dbPrimData=dbPrimData , dbFreqs=dbF, dbFingerprint=dbFp )
    else:  ## could catch low, say < 60 power level and label as "nocall" ********************************
        print("@@@@@@@ Failing save Criteria = Freq, avFre, fMaxP" ,fMaxPFreq,", ", fFreqMean," - ", fMaxP ) # Need to deal with over dropping

#     print("db Diff Spacing = ", fSpacing)

    newRow()
    addRowValue('ecode', ecode)
    addRowValue('file', file)
    addRowValue('sPitch', fMaxP )
    addRowValue('sSpacing', fFreqMean )  
    addRowValue('sDutyCycle', fMaxPFreq )
    nRowID = appendRow()
        
    return nRowID


In [ ]:
def showBirdSong(audio_file, sr_file, name = 'bird_file'):
    
    fig, ax = plt.subplots(1, figsize = (12, 4)) # 16,9 works
    fig.suptitle('Sound Waves', fontsize=16)

    librosa.display.waveplot(y = audio_file, sr = sr_file, color = "#A300F9", ax=ax);
#     librosa.display.waveplot(y = audio_cangoo, sr = sr_cangoo, color = "#4300FF", ax=ax[1])
#     librosa.display.waveplot(y = audio_haiwoo, sr = sr_haiwoo, color = "#009DFF", ax=ax[2])
#     librosa.display.waveplot(y = audio_pingro, sr = sr_pingro, color = "#00FFB0", ax=ax[3])
#     librosa.display.waveplot(y = audio_vesspa, sr = sr_vesspa, color = "#D9FF00", ax=ax[4]);

    ax.set_ylabel(name, fontsize=13)
    plt.show()
    return

In [ ]:
def audioRanging(data, nPercCutoff=20): # Returns mirror of array with range 0-
    fMax = np.max(data)
    data_range = ((data.copy())*(100/fMax))//1
    data_range_abs = abs(data_range)

    chirp_filter = (data_range_abs >= nPercCutoff) # 0 and 1
    chirp_data = chirp_filter * data_range # data below Cutoff set to 0
    fDutyCycle = round( (sum(chirp_filter) / len(chirp_filter))*100, 2 )
#     fDutyCycle = ((sum(chirp_filter) / len(chirp_filter))*10000//1)/100  #round(fSecFile,2)
    
#     if nPipeRuns < 5:
#         print(" sum(chirp_filter), len(chirp_filter), fDutyCycle  ", sum(chirp_filter), len(chirp_filter), fDutyCycle)
        
    return chirp_data, fDutyCycle, data_range  ## chirp is 0-2 with zero where under cutoff, range is 0-2

Steps for 5 second pass of just data:

1. Read whole file, store in global? - readBirdFile returns array of 5 sec clips
2. Scan file for quiet/loud spots with filter, filter to zero (may clip part of valid peaks near zero, think waves) - done in above
3. For a given 5 seconds try to find peak frequency in first 3 seconds
4. Center just before start of peak and go to end of file
5. Record fingerprint of peak+ 2 sec or so (allow for later peaks too, posibly by using rest of file)
6. Hmm, see if I can lock in some portion of the song birds with known signatures, I could then scan for only those and label rest as no song.
7. Match top 5 birds to a given 5 sec. segment, use threshold to ignore all, or keep some
8. For site 3 a post process will be needed to add all birds to the site as a whole (as whole file has single bird list)
9. For sites 1,2 the birds will need to be chosen and affixed to the right rows.  (I could find bird at a given timestamp possibly)

In [ ]:
def load_test_clip(path, start_time, duration=5):
    return librosa.load(path, offset=start_time, duration=duration, sr=nSAMPLERATE,mono=True, res_type='kaiser_fast') # [0] Kaiser_fast resampling is faster


In [ ]:

lastFile = "" 
_y_bird = []
_sr_bird = 0
def readBirdFile(file, secIteration=5.0, bIterate = True, secStart=0.0, secDur=None): # Returns array of 5 sec sections of file
    if secStart == None:
        secStart = 0.0
    secEnd = secIteration # updated later with end time if less than secIter..
    global _y_bird
    global _sr_bird
    global lastFile
    bKeepBirds = False
    bKeepFileData = len(lastFile)>5 and (lastFile.lower() == file.lower())   
#     bKeepFileData = False
    
    # Importing the file
#     if 1:
    try:
        if (bKeepFileData == False) or (len(_y_bird)<100): # if true use existing data     
            if (bPREDICTRUN == True):       
                _y_bird, _sr_bird = load_test_clip(file, start_time=secStart, duration=None)
            else:
                _y_bird, _sr_bird = load_test_clip(file, start_time=secStart, duration=None)
        else:
            print('Keeping lastData...lastFile, file',lastFile, file )
            print("From Que st, dur = ",secStart,secDur )
            print("_y_bird = ", _y_bird[:5])
            
        lastFile = file
        if (secDur != None):
            y_bird_loc = _y_bird[int(secStart*nSAMPLERATE):int((secStart+secDur)*nSAMPLERATE)] # local segment
        else:
            y_bird_loc = _y_bird[int(secStart*nSAMPLERATE):] 
            
        chirp_data,fDutyCycle,_ = audioRanging(y_bird_loc) ## filter to int 1-100 for over threshold data
        y_bird = chirp_data
        if bFILTER == True: #
            y_bird = butter_Xpass_filter(y_bird, 900, nSAMPLERATE, order=5)############ LOWPASS FILTER START level
#             y_bird = butter_lowpass_filter(y_bird, 900, nSAMPLERATE, order=5)############ LOWPASS FILTER START level            
            fMaxP, nMaxP, fMaxPFreq = getMaxP(y_bird)
            print("At ReadBirdFile finished butter filter...fr = ", fMaxPFreq)
            if fMaxPFreq <1100:
                print("## RERUN at 180 - First Freq = ", fMaxPFreq)
                y_bird = butter_Xpass_filter(y_bird, 180, nSAMPLERATE, order=5)############ LOWPASS FILTER START level
                fMaxP, nMaxP, fMaxPFreq = getMaxP(y_bird)
                
                if fMaxPFreq <500:
                    print("Second Freq = ", fMaxPFreq)
                    y_bird = butter_Xpass_filter(y_bird, 30, nSAMPLERATE, order=5)############ LOWPASS FILTER START level
                    fMaxP, nMaxP, fMaxPFreq = getMaxP(y_bird)
                    print("Third Freq = ", fMaxPFreq)
                               
        nPoints = len(y_bird)
        fSecFile = nPoints/nSAMPLERATE
#         print("WHOLE #### File max P values p,n,f , TotSecs= ",fMaxP, nMaxP, fMaxPFreq, round(fSecFile,2) )

        nTotIter = math.ceil(fSecFile / secIteration)
        if bPREDICTRUN == False:
            if bIterate == True:
                secLastIter = fSecFile % secIteration  
                if secLastIter < 3.0 and fSecFile > secIteration:
                    nTotIter = nTotIter-1 # Last iter too short, let's skip it  ### Keep if Submission file??
            else:
                nTotIter = 1
            if nTotIter > nMAXSONGITER:
                nTotIter = nMAXSONGITER # limit to this number of segments
        else:  # pred run
            bKeepBirds = nTotIter > 1 # site 3 likely
            
            
            
        y_bird_arr = np.empty((int(nTotIter), math.ceil(secIteration*nSAMPLERATE)-1),dtype=float)
        y_chirp_arr = y_bird_arr.copy()
        for nIter in range(nTotIter):
#             print("Read file, parsing - #", nIter, " - of ", nTotIter)
            if fSecFile <= secEnd:
                secEnd = fSecFile

#             print("sec Start, End = ",secStart , secEnd )
            audio_bird =  (y_bird[int(secStart*nSAMPLERATE):int(secEnd*nSAMPLERATE)-1]).copy() # 5 sec song
            audio_chirp =  (chirp_data[int(secStart*nSAMPLERATE):int(secEnd*nSAMPLERATE)-1]).copy() # 5 sec w/zeros
            if audio_bird.shape==(secIteration*nSAMPLERATE-1,):
                y_bird_arr[nIter]= audio_bird.copy()   ## Building Array of 5 second segments
                y_chirp_arr[nIter]= audio_chirp.copy()
            else:
                nShort = int((secIteration*nSAMPLERATE-1)- len(audio_bird))
                audio_bird = np.append(audio_bird, np.zeros((nShort)), axis=None)
                audio_chirp = np.append(audio_chirp, np.zeros((nShort)), axis=None)
#                 print("Finally Else case .....shape = ", audio_bird.shape)
                if audio_bird.shape==(secIteration*nSAMPLERATE-1,):
                    y_bird_arr[nIter]= audio_bird.copy()
                    y_chirp_arr[nIter]= audio_chirp.copy()
#                     print("Odd shape - success!")
            secStart = secEnd
            secEnd = secEnd + secIteration
    except Exception as e:
        print("###### ERROR READING - file - ", file)
        print("EXCEPTION  = ",e)
        return [], [], -1, False
#     except:
#         print("###### ERROR READING - file - ", file)
# #         bRUNNING = False
        
#     print("y_bird_arr = ", y_bird_arr)

    return y_bird_arr, y_chirp_arr, fDutyCycle, bKeepBirds

In [ ]:

nPipeRuns = 0
def birdPipeline( objSong=newToProcessRow ):   #file, idTrain = ""):
    global nPipeRuns
    global audio_bird_arr
    global audio_chirp_arr
    global fDutyCycle
    nPipeRuns +=1
#     global train
    global bPREDICTRUN
#     print("Obj = ", objSong)
#     print("train = " , train[:3])

    file = objSong.file
    idTrain = objSong.FileRefId
    secStart = objSong.startSec
    secDur = objSong.duration
    nIter = objSong.nIter
    nLoc = objSong.nLoc

    if bPREDICTRUN == False:
        ecode = train.iloc[objSong['nIndex'],:]['ebird_code'] # keeps not changing ### ERROR HERE on Submit code
        print("Bird Pipeline --- eCode  = ", ecode," -  pipe run #", nPipeRuns,"" )
    else:
        ecode = 'unknown'  #enumECodes['nocall'] #262 #test.iloc[objSong['nIndex'],:]['ebird_code']
        print("Bird Pipeline --- eCode  = Unk, file = ",file, " -  pipe run #", nPipeRuns,"" )
    #print("string ecode = ", ecode)

   
#     if nIter < 1:
    # Importing the file
    audio_bird_arr,audio_chirp_arr,fDutyCycle, bKeepBirds = readBirdFile(file, secIteration=5.0, bIterate = True, secStart=secStart, secDur=secDur) ##### READ FILE LOOP
    if fDutyCycle < 0:
        return nPipeRuns # ERROR on read try
    if bKeepBirds:
        print("KEEP BIRDS set #############################")
        print("From Que st, dur = ",secStart,secDur )
    
    nIters = audio_bird_arr.shape[0]
#     print("number of iterations = ", nIters)
    audio_bird = audio_bird_arr[0]
    audio_chirp = audio_chirp_arr[0]
      
    bShow = not bSUBMITRUN #bFASTRUN    
    for nIter in range(nIters):  
        audio_bird = audio_bird_arr[nIter]
        audio_chirp = audio_chirp_arr[nIter]
        if bShow == True and nPipeRuns<5:
            showBirdSong(audio_bird, nSAMPLERATE, name = idTrain)
#         print("found ecode = ", ecode," - loopSongID = ", objSong.nIndex)
        ### Below is fed 5 sec sections of data, filename passed is only for reference
        calcStats(audio_bird, audio_chirp, filename=file, ecode = ecode, nIndex = objSong.nIndex,fDutyCycle=fDutyCycle, nIter=nIter )  
    
    
    return nPipeRuns


In [ ]:
#CPU count
nCPUs = cpu_count()
print("cpu_count = ", nCPUs)

In [ ]:
nTotFiles = 0

def addBirdFileToQ(file, idTrain = "",fStartSec = 0.0,fDur = None, nIter=0, nLoc=1): #,'startSec', 'duration' 
    global qToProcess
    global nTotFiles
    
    if len(file)>5:
        file_short = os.path.basename(file)
    else:
        file_short = "NA"
        return -1
    nTotFiles +=1
    print("Added to q st, dur = ",fStartSec,fDur )  
    newToProcessRow = pd.Series(data=qToProcess_row, index=qToProcess_columns) #, 'nIter', 'nLoc'
    newToProcessRow['nIndex'] = idTrain
    newToProcessRow['file'] = file
    newToProcessRow['FileRefId'] = str(file_short) +"_" + str(idTrain)
    newToProcessRow['nStatus'] = 0
    newToProcessRow['startSec'] = fStartSec
    newToProcessRow['duration'] = fDur
    newToProcessRow['nIter'] = int(nIter)
    newToProcessRow['nLoc'] = int(nLoc)
    qToProcess.put(newToProcessRow)
    qSize = qToProcess.qsize()

    print("To Process Que size = ", qSize)
    return nTotFiles

In [ ]:


def workerFileQ():
    items = []
    bTask = False
    global bRUNNING
    bRUNNING = True
    print("Worker Thread Starting **********")
    while bRUNNING == True:
        bTask = False

        lSong = qToProcess.get()
        bTask = True
          
        nSongs = birdPipeline(lSong)  #.file, lSong.FileRefId)
        qToProcess.task_done()
#         print("Number = " + str(nSongs) + ", - Finished " , lSong.FileRefId," ")
        if (qToProcess.qsize() < 1):
            time.sleep(1000/1000)
        
        if (qToProcess.qsize() < 1):
#             print("Q is done I believe..")
            time.sleep(3000/1000)
            if (qToProcess.qsize() < 1):
                print("Task thread timeout to finish....")
                break
    print("Worker Thread Closing.... XXXXXXXXXXX")
        

In [ ]:
def addToSongQue(file, ecode='None'):
    newSongsRow = pd.Series(data=queSongs_row, index=queSongs_columns)
    newSongsRow['file'] = file
    newSongsRow['ecode'] = ecode
    return

# MAIN BIRD LOOP 
# =======================

In [ ]:
# dbEbirdCodes[120]

In [ ]:
# loopSongIDs = train[train['ebird_code'] == dbEbirdCodes[186]][73:101].index
# for loopSongID in loopSongIDs:
#     print("song ids = ",loopSongID)
    
# print("Done")

In [ ]:
##### code = ""
def birdLoop():
    
    startTime = time.time()
    avgTime = 0.0
    
    bSpecialList = False
    bSList = [67,136]
    #### FOR ALL run, index,nMax, perCode => 0, 999, variable by slice
    nStartCodeIndex = 15 # lotduc 147 i believe # Full missing: 76,82, 96, 120
    nMaxCodes = 2     # of Species tested
    nMaxPerCode = 1    # song files per Species
    nMaxPerHardCode = nMaxPerCode*3
    global dbEbirdCodes
    global bRUNPARALLEL
    global nCPUs
    initRun()
    
    bRUNPARALLEL = False
    global code
    global loopSongID
    loopSongID = 0
    print("Init done, starting filling que....")

    if True:
        nLoops = 0
        nRunSongs = 0
        for code in dbEbirdCodes:
            nEcode = enumECodes[code]
#             if (bSpecialList == True) and (code in bSList) # if item in my_list:
            if ((bSpecialList == True) and (nEcode in bSList)) or (nLoops >= nStartCodeIndex and nLoops < nMaxCodes+nStartCodeIndex):
                
                if nEcode in [67, 76, 77, 82, 96, 120, 136, 152]: # 67, 136 missing from full set
                    nMaxPer = nMaxPerCode  ###### nMaxPerHardCode
                    print("Doing harder code:....", code)
                else:
                    nMaxPer = nMaxPerCode
#                     print("NOT - extra codes....", code)
                bAllSongs = False
                if bAllSongs == True:
                    loopSongIDs = train[train['ebird_code'] == code][50:nMaxPerCode].index #### MODIFY here for partial sets -50:nMax...
#                     loopSongIDs = train[train['ebird_code'] == code][25:].index
                else:
                    loopSongIDs = train[train['ebird_code'] == code].sample(nMaxPer, replace=True).index

                for loopSongID in loopSongIDs:
                    bQrows = True
                    if bQrows == True:
                        rowSong = train.iloc[int(loopSongID) ]
                        loopSong = rowSong['full_path']
                        fDur = rowSong['duration']
#                     Qrows, nQrows = createQrow(train, int(loopSongID) ) ## added nIter, seconds, path
                        Qrows, nQrows = trainCreateQrow(loopSong, duration=fDur)
                        for nRowId in range(len(Qrows)):
                            fStartSec = int(Qrows.loc[nRowId,['seconds']]) - 5.0 #0.0 ## not specified in test
                            fDur = 5.0
                            nIter = int(Qrows.loc[nRowId,['nIter']]) 
#                             print("loopSongID: fSsec, nIter = ",loopSongID,": ",fStartSec, nIter )
                            addBirdFileToQ(loopSong, idTrain=loopSongID,fStartSec = fStartSec,fDur = fDur,nIter=nIter)                            
                    
                    else:
                        rowSong = train.iloc[int(loopSongID) ]
                        loopSong = rowSong['full_path']
                        fStartSec = 0.0 ## not specified in test
                        fDur = rowSong['duration']
    
                        addBirdFileToQ(loopSong, idTrain=loopSongID,fStartSec = fStartSec,fDur = fDur)
                    
                    nRunSongs += 1
                    totTime = (time.time() - startTime)
                    avgTime = totTime/nRunSongs
#                     print("Average, Total Loop Time = ", avgTime," , ", totTime)
            nLoops += 1  
   
            
    # block until all tasks are done
    qToProcess.join()
    print('All work completed')
    bRUNNING = False

    endTime = time.time()
    print("Total time = {:.4f}".format(endTime-startTime))
    print("Total time = ", (endTime-startTime))

    return

bad file lotduc/XC195038.mp3

In [ ]:
bPREDICTRUN = False
bSkipBLoop = True
if (bPREDICTRUN == False) and (bSkipBLoop == False):
    birdLoop() # Skip usually
    saveData(bFinderDb, strFileName = "Bproj_data")



In [ ]:
birdDb.head(2)

Results close, but only good for first row of file currently I believe, but getting close.

In [ ]:
# birdDb.tail(20)

In [ ]:
bFinderDb.head(30)

In [ ]:
bFinderDb.tail(30)

# Model NN

## StratifiedKfold

https://towardsdatascience.com/writing-your-first-neural-net-in-less-than-30-lines-of-code-with-keras-18e160a35502

This might be easier....

In [ ]:
from keras import models
from keras import layers
from keras.utils import to_categorical

nTrainCol = 27
# (train_images, train_labels), (test_images, test_labels) = mnist.load_data()

In [ ]:
bUseDisk = True ## using last data set vs new for now
if bUseDisk == True:
#     train_all = train_part1.append(train_part2)
#     train_all = train_all.append(train_part3)
#     train_all = train_all.append(train_part4)
#     train_all = train_all.append(train_part5)
#     train_all = train_all.append(train_part6)
#     train_all = train_all.append(train_part7)
#     train_all = train_all.append(train_part8)
#     train_all = train_all.append(train_part9)
    
    train_all = train_part1b.append(train_part2b)
    train_all = train_all.append(train_part3b)
    train_all = train_all.append(train_part4b)
    train_all = train_all.append(train_part5b)
    train_all = train_all.append(train_part6b)
    train_all = train_all.append(train_part7b)
#     train_all = train_all.append(train_part8)
#     train_all = train_all.append(train_part9)
    # train_part5
else:
    train_all = bFinderDb.copy()

train_all.head()

In [ ]:
# train_all.loc[(train_all['ecode'] ==  enumECodes['amebit'])  ]

In [ ]:
countTrainBirds(train_all)

In [ ]:
print("Shape Before Filter = ", train_all.shape)
train_all_filtered = train_all.loc[(train_all['ecode'] <  enumECodes['nocall']) & (train_all['primSl'] >  50.00) ]# was != working
nECodes = len(train_all_filtered['ecode'].unique())
# if nECodes < nEbirdCodes-2:
#     print("1 - too few birds x vs y",nECodes, nEbirdCodes-1 )
#     train_all_filtered = train_all.loc[(train_all['ecode'] !=  enumECodes['nocall']) & (train_all['primSl'] >  70) ] # 87.83- Works
# nECodes = len(train_all_filtered['ecode'].unique())
# if nECodes < nEbirdCodes-2:
#     print("2 - too few birds x vs y",nECodes, nEbirdCodes-1 )
#     train_all_filtered = train_all.loc[(train_all['ecode'] !=  enumECodes['nocall']) & (train_all['primSl'] >  65) ]   
nECodes = len(train_all_filtered['ecode'].unique())
print("====Compare birds x vs y",nECodes, nEbirdCodes-1 )
# Note above use & opperator inside loc for compare vs 'and' in normal python
# train_all_filtered = train_all
print("Shape after nocall removed = ", train_all_filtered.shape)
train_all_filtered.head()

In [ ]:
# train_all_filtered.sort_values(['ecode'], ascending=False, inplace=True)
# train_all_filtered2 = train_all_filtered.copy()
# dfNUnique = train_all_filtered2['ecode'].value_counts()


In [ ]:
# print(dfNUnique[-10:])

In [ ]:
## Filter out all but best 20 songs
train_all_filtered.sort_values(['primSl'], ascending=False, inplace=True) # .sort_values(['primSl'], ascending=False, inplace=True)
train_all_filtered2 = train_all_filtered.copy()
dbTrainECodes = train_all_filtered2['ecode'].unique()
print("Train Ecodes unique = ", len(dbTrainECodes))
drop_indexes = []
nDropThreshold = 100
for nCode in dbTrainECodes:
    ecode_best = train_all_filtered2.loc[(train_all_filtered2['ecode'] == nCode )].index
#     bestI = (train_all_filtered2[ecode_best]).sort_values(['primSl'], ascending=False).index
#     ecode_best.sort_values(['ecode'], ascending=False, inplace=True)
    
#     data.sort_values("Name", axis = 0, ascending = True, 
#                  inplace = True, na_position ='last') 

#     .sort_values('ecode', ascending=False, inplace=True)
#     if ecode_best != None:
    nLen = len(ecode_best)
#     print("code = ", enumECodesRev[nCode]," - , actual = ",nLen )
#     else:
#         nLen = 0
#         print("None at - ", nCode,ecode_best )
#     DataFrame.sort_values(by, axis=0, ascending=True, inplace=False, kind='quicksort', na_position='last')
    if nLen > nDropThreshold:
        ecode_drop = ecode_best[:nDropThreshold]
        drop_indexes.extend(ecode_drop)
#         train_all_filtered2.drop(ecode_drop, inplace=True) ## comment/uncomment to DROP ### NOT WORKING CORRECT DROPPING WRONG ITEMS
    else:
        print("code = ", enumECodesRev[nCode]," - short of ",nDropThreshold,", actual = ",nLen )
#     ecode_best = train_all_filtered2.loc[(train_all_filtered2['ecode'] == nCode )].index
#     nLen = len(ecode_best)
#     print("AFTER code = ", enumECodesRev[nCode]," -  of 20, actual = ",nLen ) 
print("Shape after below ",nDropThreshold," removal (of ",train_all_filtered.shape[0],")= ", train_all_filtered2.shape) 


# print("Length of drops = ", len(drop_indexes))
# train_all_filtered2 = train_all_filtered2.iloc[drop_indexes]
# # train_all_filtered2.drop(drop_indexes, inplace=True, )   
# print("Shape after below 20 removal (of 58413)= ", train_all_filtered2.shape)   
# len(train_all_filtered2['ecode'].unique() ) 
    

In [ ]:
train_all_filtered2.head(30)

In [ ]:
#  train , test is our sets here

# trainData = train_all_filtered2.loc[:,'primFr':].fillna(0)
# trainData = train_all_filtered2.loc[:,['avgFr','dCycle','primSl']].fillna(0) ## Limited columns 
trainData = train_all_filtered2.loc[:,['fFundFreq','dCycle','primSl']].fillna(0) ## Limited columns avgFr ## 20% plus!! at 15 sample rate

# trainData = trainData['primSl','dCycle','fFundFreq','nFFmult']
# train = train_full1.loc[:,'primFr':'dCycle'] 
trainData_labels = train_all_filtered2['ecode'].fillna(0)

In [ ]:
trainData.head()

# Model includes - may move up 

In [ ]:
# https://scikit-learn.org/stable/auto_examples/classification/plot_classifier_comparison.html#sphx-glr-auto-examples-classification-plot-classifier-comparison-py
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.ensemble import GradientBoostingClassifier # XGB trying to add

from sklearn.preprocessing import StandardScaler
from sklearn import model_selection
#  # from Titanic example https://www.kaggle.com/kaerunantoka/titanic-pytorch-nn-tutorial

from sklearn.preprocessing import LabelEncoder

# from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score



## Scaling

In [ ]:
numeric_cols = list(set(trainData.columns) )
# train[numeric_cols] = train[numeric_cols].fillna(0)
print('scaling numeric columns')

scaler = StandardScaler()
trainData[numeric_cols] = scaler.fit_transform(trainData[numeric_cols])
# trainData = scaler.fit_transform(trainData)
# train = scaler.fit_transform(train)
# train_labels = scaler.fit_transform(train_labels)

trainData = trainData.astype('float32')
trainData_labels = trainData_labels.astype('float32')

In [ ]:
from random import random
# Train / Validation Split of test data
fVal_size = 0.20
seed = int((random()*1000)//1) # 7
print("random seed is  = ", seed)
scoring = 'accuracy'
X = trainData.copy()
Y = trainData_labels.copy()
X_train,X_val, Y_train, Y_val = model_selection.train_test_split( X, Y,
                                                                test_size=fVal_size,
                                                                random_state = seed)



In [ ]:
for nMax in range(150):
    nECodes = len(Y_train.unique())
    if nECodes < nEbirdCodes-2: # leave room for nocall not there for now
        print("Running split again to get all birds....Run, ecodes # ", nMax, nECodes)
        seed = int((random()*1000)//1)
        X_train,X_val, Y_train, Y_val = model_selection.train_test_split( X, Y, stratify=Y, 
                                                                test_size=fVal_size,
                                                                random_state = seed)
        
#         X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)
    else:
        print("Test split Good, ecodes count # ", nECodes)
        break
        


In [ ]:
# X_train.head(3)
X_train[:1]
X_train.shape

In [ ]:
Y_train.head(1)

In [ ]:
import xgboost as xgb
# from xgboost import XGBClassifier
# from sklearn.metrics import accuracy_score

## Algorithm compare
models = []
# models.append(('LR', LogisticRegression() ))
# models.append(('LDA', LinearDiscriminantAnalysis() ))
# models.append(('KNN', KNeighborsClassifier() ))
models.append(('CART', DecisionTreeClassifier(min_samples_leaf = 10) ))
# models.append(('NB', GaussianNB() ))
# models.append(('SVM', SVC(kernel='linear') )) ## Very time intensive
#  self.model=XGBClassifier(n_estimators=1000,learning_rate=0.02,**params)
# models.append(('XGB-CL', xgb.XGBClassifier(random_state=1,learning_rate=0.01, max_depth=10, verbosity=100) )) #GradientBoostingClassifier(random_state=0)

# models.append(('GB-CL', GradientBoostingClassifier(random_state=1,learning_rate=0.01, max_depth=10) )) #GradientBoostingClassifier(random_state=0)

# import xgboost as xgb
# model=xgb.XGBClassifier(random_state=1,learning_rate=0.01)
# model.fit(x_train, y_train)
# model.score(x_test,y_test)

# compare models
results = []
names = []
bFindBestModel = True
if bFindBestModel == True:
    for name, model in models:
        kfold = model_selection.KFold(n_splits=20, random_state=seed)
        cv_results = model_selection.cross_val_score(model, X_train.values,
                                                     Y_train, cv=kfold, scoring=scoring,
                                                     n_jobs=-1, verbose=20)  # -1 is all processors, v=0 none, 10 is all iterations
        results.append(cv_results)
        names.append(name)
        print("",name," :: Mean =", cv_results.mean(), ", STD =", cv_results.std() )


     CART  :: Mean = 0.08211292076825748 , STD = 0.002850388649308943 - at 70


    >  LR  :: Mean = 0.09053031379320882 , STD = 0.004155416885909685 at 80
    >  LDA  :: Mean = 0.07672898283522037 , STD = 0.003380384083820006
    >  KNN  :: Mean = 0.07647277369049998 , STD = 0.004118654987786331
    >  CART  :: Mean = 0.09605785587418034 , STD = 0.004120997742153965
    >  NB  :: Mean = 0.07365413802698048 , STD = 0.005369852575328259
    
      CART  :: Mean = 0.0957646280195982 , STD = 0.0038940212223722973 - at 85
    
     CART  :: Mean = 0.09983974688631879 , STD = 0.007691210144059721 - at 87
     
     CART  :: Mean = 0.10447520432755628 , STD = 0.010142759083692596 - AT 87.5
     
     CART  :: Mean = 0.10235265727277416 , STD = 0.00804816806714718- AT 87.83
     CART  :: Mean = 0.10802477695100365 , STD = 0.007889066415571452 - RUN 2


In [ ]:
train.shape

In [ ]:
bXGB = False
if bXGB == True:
    # importing required libraries
    from xgboost import XGBClassifier
#     from sklearn.metrics import accuracy_score
 
    # seperate the independent and target variable on testing data
#     test_x = test_data.drop(columns=['Survived'],axis=1)
#     test_y = test_data['Survived']
#     X_train,X_val, Y_train, Y_val = model_selection.train_test_split
    
    model_xgb = XGBClassifier(random_state=1,learning_rate=0.01, max_depth=10, verbosity=1)
 
    # fit the model with the training data
    model_xgb.fit(X_train,Y_train)
 
 
    # predict the target on the train dataset
    predict_train = model_xgb.predict(X_train)
    print('\nTarget on train data',predict_train) 
 
    # Accuray Score on train dataset
    accuracy_train = accuracy_score(Y_train,predict_train)
    print('\naccuracy_score on train dataset : ', accuracy_train)
 
    # predict the target on the test dataset
    predict_test = model_xgb.predict(X_val)
    print('\nTarget on test data',predict_test) 
 
    # Accuracy Score on test dataset
    accuracy_test = accuracy_score(Y_val,predict_test)
    print('\naccuracy_score on test dataset : ', accuracy_test)
    
# accuracy_score on train dataset :  0.256266850865661
# accuracy_score on test dataset :  0.08021390374331551

In [ ]:
# linear regression feature importance
from sklearn.datasets import make_regression
from sklearn.linear_model import LinearRegression
from matplotlib import pyplot
# define dataset
# X, y = make_regression(n_samples=1280, n_features=27, n_informative=5, random_state=1)
bBuildModel = False # for submit set to false to save/load
if bBuildModel == True:
# define the model svm = svm.SVC(kernel='linear')
#     model = SVC(kernel='linear', probability=True, verbose=True) # default LR = 0.01
# model = LinearRegression() #
    model = DecisionTreeClassifier(min_samples_leaf = 10) #
    # fit the model
    model.fit(X_train, Y_train)
    # save model
    pickle.dump(model, open(model_filename, 'wb'))
else:
# load the model from disk
    model = pickle.load(open(model_filename, 'rb'))
#     result = loaded_model.score(X_test, Y_test)

    
bBuildModel = True    
if bBuildModel == True:

#   fit(
#     x=None, y=None, batch_size=None, epochs=1, verbose=1, callbacks=None,
#     validation_split=0.0, validation_data=None, shuffle=True, class_weight=None,
#     sample_weight=None, initial_epoch=0, steps_per_epoch=None,
#     validation_steps=None, validation_batch_size=None, validation_freq=1,
#     max_queue_size=10, workers=1, use_multiprocessing=False  )
    
# get importance
# importance = model.coef_ # LinRegr..
    importance = model.feature_importances_  # CART
#     importance = abs(model.coef_[0]) # SVM..


# summarize feature importance
    for i,v in enumerate(importance):
        print('Feature: %0d, Score: %.5f' % (i,v))
# plot feature importance
    pyplot.bar([x for x in range(len(importance))], importance)
    pyplot.show()
    
    

In [ ]:
# importance[1]

In [ ]:
def perc2Pred(data, fThresh=3.0, nTopBirds=1):
#     print(data[:5])
    top_n_pred = np.argsort(data, axis=1)[:,-nTopBirds :]
    class_labels = model.classes_
    birdPr = class_labels[top_n_pred][:,0]
    for index in range(len(birdPr)):
        fPercentage = data[index,top_n_pred[index]]*100
#         print("Percentage = ", fPercentage)
        if fPercentage<fThresh: # Theshold
            birdPr[index] = enumECodes['nocall']
    return birdPr

In [ ]:
# predData = dbPrep4Predict(bFinderDb)
# #     SVMpreds = model.predict(predData)
# SVMpreds = model.predict_proba(predData)
# SVMpreds2 = perc2Pred(SVMpreds, fThresh=3)#fThresh=5.0, nTopBirds=1

# print(SVMpreds2[:25])

In [ ]:
# perc2Pred(SVMpreds,35.0)


In [ ]:
if bBuildModel == True:
# model = DecisionTreeClassifier() #
# fit the model
# model.fit(X_train, Y_train)
    SVMpreds = model.predict_proba(X_val) # predict with probability
    SVMperc = perc2Pred(SVMpreds,3.0) # probability cutoff
# SVMpreds = model.predict(X_val)



In [ ]:
if bBuildModel == True:
    print(accuracy_score(Y_val,SVMperc ))
    print(confusion_matrix(Y_val,SVMperc ))
    print(classification_report(Y_val,SVMperc ))


In [ ]:
# SVMperc = perc2Pred(SVMpreds)
# SVMperc

In [ ]:
# SVMpreds.shape
# n=1
# top_n_pred = np.argsort(SVMpreds, axis=1)[:,-n :]
# class_labels = model.classes_

In [ ]:
# SVMperc[:20]
# SVMVals = pd.value_counts(SVMperc, ascending=False) # was SVMpreds

# SVMVals_high = SVMVals[:10]


In [ ]:
# SVMVals

In [ ]:
# print(zip(*SVMVals)[0]) #zip(*tuples)[0]

# summarize predictions
# for i,v in enumerate(SVMVals):
#     if v>10:
#         print('Feature: %0d, Score: %.5f' % (i,v))
# # plot preds count
# pyplot.bar([x for x in range(len(SVMVals))], SVMVals)

# pyplot.show()

## Submission


In [ ]:
# def fiveSecPredict(data):
#     topBirds = []
#     topBirdPreds = []
    
#     return topBirds,topBirdPreds

In [ ]:
# def runPredict():
#     # 3 sites, first 2 by 5 second labels, 3rd by whole file labels
#     # How about  second intervals, get top 5 predicted birds in each 5 secs
    
    
#     return

# Submission parsing

maybe use, if sample times given...

Steps:
Get File, get total seconds
Get file parsing info
if not info parse every 5 seconds


In [ ]:
test.head(2)

In [ ]:
# def splitLargeFile(large_clip):
#     clip_arr = []
#     nPoints = len(large_clip)
#     fSecFile = nPoints/nSAMPLERATE 
#     secIteration = 5.0
    
#     secLastIter = fSecFile % secIteration
#     nTotIter = math.ceil(fSecFile / secIteration)
#     if secLastIter < 2.0 and fSecFile > secIteration:
#             nTotIter = nTotIter-1 # Last iter too short, let's skip it  ### Keep if Submission file??
#     y_bird_arr = np.empty((int(nTotIter), math.ceil(secIteration*nSAMPLERATE)-1),dtype=float)
#     for nIter in range(nTotIter):
#         if fSecFile <= secEnd:
#             secEnd = fSecFile
#         audio_bird =  (large_clip[int(secStart*nSAMPLERATE):int(secEnd*nSAMPLERATE)-1]).copy()
#         clip_arr[nIter]= audio_bird.copy()   ## Building Array of 5 second segments
        
#     return clip_arr

In [ ]:
# def make_prediction(sound_clip, birds=dbEbirdCodes):
#     return np.random.choice(birds)

def make_prediction(sound_clip, birds=dbEbirdCodes):
    
#     addBirdFileToQ(loopSong, idTrain=loopSongID)
    
    if np.random.randint(0,9) % 5 == 0:
        return np.random.choice(birds)
    return "nocall" # near half "nocall", so just guessing at ~10%

In [ ]:

def dbPrep4Predict(data):
    if len(data) < 1:
        print("ERROR - dbPrep4Predict - empty data array.")
        return []
     # bFinderDb.head(30)
    numeric_cols = list(set(data.columns) )
#     data1 = data.loc[:,'primFr':].fillna(0).copy() # 27 columns
    data1 = data.loc[:,['fFundFreq','dCycle','primSl']].fillna(0).copy() # 3 columns

#     print(data1[:5])
#     data1[numeric_cols] = scaler.fit_transform(data1[numeric_cols])
    data1 = scaler.fit_transform(data1)
#     print(data1)
    return data1


# pred = model.predict(test1)[0]     #make_prediction(sound_clip)
# print("Pred, p = ",[bFinderDb[index]], pred )

In [ ]:
# bFinderDb

In [ ]:
# predData = dbPrep4Predict(bFinderDb)
# # print(predData[:5])
# #     SVMpreds = model.predict(predData)
# SVMpreds = model.predict_proba(predData)
# # print(SVMpreds[:5])
# SVMpreds2 = perc2Pred(SVMpreds,30.0)
# print(SVMpreds2[:5])

In [ ]:
# numeric_cols=list(set(bFinderDb.columns) )

In [ ]:
# dbPrep4Predict(bFinderDb)
                    
#                     loopSong = train.iloc[int(loopSongID) ]['full_path']
                   

In [ ]:

def Run_Predict(test):  ## enter with test = test db to use
    global dbEbirdCodes
    global bRUNPARALLEL
    global nCPUs
    initRun()
    global bPREDICTRUN
    bPREDICTRUN = True
    global SVMpreds # just for debugging
    bRUNPARALLEL = False
    global code
    global loopSongID
    loopSongID = 0
    print("Init done, starting filling que....")
    bQueing = True
    startTime = time.time()

    try:  # Comment out top and bottom to allow error propogation
    # if True:
        preds = []
        nCount=0
        for index, row in test.iterrows():
            loopSongID = index # same as ID in training
            nCount +=1
            bTestPredict = False
#             if (nCount >= 5) and ( (bSUBMITRUN == False) or (bTestPredict == True) ):
#                 qToProcess.join()
#                 print("Exit after quick test of 5")
#                 continue
            
            
            # Get test row information
            site = row['site']
            start_time = row['seconds'] - 5.0
            row_id = row['row_id']
            audio_id = row['audio_id']
            row_path = row['path']
  
            print("count = #", index, ", start, end = ", start_time, ", ", row['seconds'], " ####### " )
            qToProcess.join()
            # Get the test sound clip
            if site == 'site_1' or site == 'site_2':
                fDur = 5.0
                addBirdFileToQ(row_path, idTrain=loopSongID,fStartSec = start_time,fDur = fDur)
#                 sound_clip = load_test_clip(row_path, start_time)
                print("===== site = ", site, " - ", (audio_id + '.mp3'), " - 5 sec")
                if nCount < 3:
                    print("+++++ path = ", (TEST_FOLDER + audio_id + '.mp3') )
            else:
                addBirdFileToQ(row_path, idTrain=loopSongID,fStartSec = 0.0,fDur = None)
#                 sound_clip = load_test_clip(TEST_FOLDER + audio_id + '.mp3', 0, duration=None)
                print("site = ", site, " - ", (audio_id + '.mp3'), " - No LIMIT")
            



    except Exception as e:
        preds = pd.read_csv('../input/birdsong-recognition/sample_submission.csv')
        print("EXCEPTION -- count = ", nCount," - ", row_id, audio_id)
        print("EXCEPTION  = ",e)
        predDF = pd.DataFrame(preds, columns=['row_id', 'birds'])
        bRUNNING = False
        return predDF
       
            
    # block until all tasks are done
    qToProcess.join()
    print('All work completed')
    bRUNNING = False
    
    if bBuildModel == True:
        predData = dbPrep4Predict(bFinderDb)
#     SVMpreds = model.predict(predData)
        SVMpreds = model.predict_proba(predData)
        SVMpreds = perc2Pred(SVMpreds, fThresh=5.0)#fThresh=5.0, nTopBirds=1
        print(SVMpreds[:15])
    
        predBirds=[]
        nPreds = len(SVMpreds)
        for index in range(nPreds):
            predBirds = np.append(predBirds,enumECodesRev[int(SVMpreds[index])])

        print("Shape compare test vs predBirds = ", test.shape[0]," vs ", predBirds.shape[0])
    else:
        predBirds=[]
        nPreds = len(bFinderDb)
        print("No BUILD output....")
        strPreds=""
        for index,row in bFinderDb.iterrows():
            if row['ecode'] == enumECodes['nocall']:
                predBirds = np.append(predBirds,'nocall')
            else:  
#                 predBirds = np.append(predBirds,'amecro')
                if int(row['nSegm']) < 1: # for now ONLY catch first bird segm. in area 3
                    predBirds = np.append(predBirds,enumECodesRev[ row['ecode'] ]) # Reverse lookup string
        
#         for index in range(nPreds):
#             predBirds = np.append(predBirds,enumECodesRev[int(SVMpreds[index])])
            

#         print("Shape compare test vs predBirds = ", test.shape[0]," vs ", predBirds.shape[0])
        
    preds = pd.DataFrame({'row_id':test['row_id'][:nPreds], 'birds': predBirds[:nPreds] } )
    
    predDF = pd.DataFrame(preds, columns=['row_id', 'birds'])

    
    endTime = time.time()
    print("Total time = {:.4f}".format(endTime-startTime))
    print("Total time = ", (endTime-startTime))
    return predDF

In [ ]:
bTestPredict = False
# bPREDICTRUN = True
if bPREDICTRUN == True or bTestPredict == True:
    preds = Run_Predict(test)
    saveData(bFinderDb, strFileName = "Bproj_data2")


In [ ]:
bFinderDb.shape

In [ ]:
if bPREDICTRUN == True or bTestPredict == True:
    preds.head(-30)

In [ ]:
# preds
# bFinderDb[:25]


In [ ]:
bFinderDb[-30:]



In [ ]:

# bFinderDb[-50:]


In [ ]:

# predData = dbPrep4Predict(bFinderDb)
# #     SVMpreds = model.predict(predData)
# SVMpreds = model.predict_proba(predData)
# SVMpreds = perc2Pred(SVMpreds, fThresh=1.99)#fThresh=5.0, nTopBirds=1

# print(SVMpreds[:5])
    
# predBirds=[]
# for index in range(len(test)):
#     predBirds = np.append(predBirds,enumECodesRev[int(SVMpreds[index])])

# preds = pd.DataFrame({'row_id':test['row_id'], 'birds': predBirds } )
    
# predDF = pd.DataFrame(preds, columns=['row_id', 'birds'])


In [ ]:

# frame = { 'Author': auth_series, 'Article': article_series }  
# result = pd.DataFrame(frame) 
# predBirds=[]
# for index in range(len(test)):
#     predBirds = np.append(predBirds,enumECodesRev[int(SVMpreds[index])])

# preds = pd.DataFrame({'row_id':test['row_id'], 'birds': predBirds } )
# preds.head()

In [ ]:
# key = next(key for key, value in dd.items() if value == 'value')

# predBirds=[]
# for index in range(len(SVMpreds)):
#     predBirds = np.append(predBirds,enumECodesRev[int(SVMpreds[index])])


# # SVMpreds = int(value)   for value in SVMpreds[:10]
# predBirds

In [ ]:

# def getBFile():
#     return

# def getBFileSplits():
#     return

# def buildSplitLabels():
#     return

# if sample.site in ['site_1', 'site_2']:
#     start, end = sr * (sample.seconds - 5), sr * (sample.seconds)
#     y = y[int(start):int(end)]
# else:
#     start, end = sr * (0), sr * (5)
#     y = y[int(start):int(end)]

In [ ]:
# #  train , test is our sets here

# train = train_all.loc[:,'primFr':].fillna(0)
# # train = train_full1.loc[:,'primFr':'dCycle'] 
# train_labels = train_all['ecode'].fillna(0)


# numeric_cols = list(set(train.columns) )
# # train[numeric_cols] = train[numeric_cols].fillna(0)
# print('scaling numeric columns')

# scaler = StandardScaler()
# train[numeric_cols] = scaler.fit_transform(train[numeric_cols])
# # train = scaler.fit_transform(train)
# # train_labels = scaler.fit_transform(train_labels)

# train = train.astype('float32')
# train_labels = train_labels.astype('float32')

In [ ]:
if bPREDICTRUN == True:
    saveSubmission(preds)

In [ ]:
if bPREDICTRUN == True:
    preds.head(20)

In [ ]:
train_all_filtered2.sort_values(['ecode'], ascending=True, inplace=True)
train_all_filtered2.head(25)

In [ ]:
groupSummary = train_all_filtered2.groupby(by=["ecode"])
groupSummary.head(3)

nCnt = 0
for name, group in groupSummary:
    nCnt +=1
    if nCnt % 100 == 0:
        print("name, group in groupSummary = ", name," - ", enumECodesRev[name])  #, group)
        bPrint = True
  
        for row_index, row in group.iterrows():
            if bPrint:
                print(row[3])
                print(row_index)
                bPrint = False

# df = train_all_filtered2.groupby('ecode').agg(lambda x: ','.join(x))
# for name in df.index:
#     print(name)
#     print(df.loc[name])
    


In [ ]:
#PEP-8 naming conventions
# variable_one_name = 1  # same for functions
# class CamelCaseStyle(object):  # same for TypeVariables
# This is PEP 8 conforming comment

"""Multiline Function descriptions.  First line is summary. 

This function takes a pandas dataframe with time series as the index, 
and calculates the total sum, aggregated by month. 
"""

# CONSTANT_ONE = 99
# def f(x): return 2*x   insead of ...   f = lambda x: 2*x   # single line functions

In [ ]:
def app_imports(): # call in public space, regardless will import to global space
    import numpy as np 
    import pandas as pd 

    import os

    import IPython.display as ipd
    pd.set_option('max_columns', 35)
    import matplotlib.pyplot as plt
    %matplotlib inline
    import seaborn as sns
    import pickle # serializing models  
    # save model
    # pickle.dump(model, open(filename, 'wb'))
    # load the model from disk
    # loaded_model = pickle.load(open(filename, 'rb'))
    # result = loaded_model.score(X_test, Y_test)

    import shutil
    # shutil.copy2('/src/dir/file.ext', '/dst/dir/newname.ext') # complete target filename given
    # shutil.copy2('/src/file.ext', '/dst/dir') # target filename is /dst/dir/file.ext
    # Imports


    import matplotlib.image as mpimg
    from matplotlib.offsetbox import AnnotationBbox, OffsetImage

    # Map 1 library
    import plotly.express as px

    # Map 2 libraries
    import descartes
    import geopandas as gpd
    from shapely.geometry import Point, Polygon

    #install numba==0.48
    # above is hack to not hit 0.5 removal of decorators

    import numba
    #==0.48
    import numba.decorators

    import sklearn

    import warnings
    warnings.filterwarnings('ignore')
    
    # Sound Imports
    import librosa
    import librosa.display
    from joblib import Parallel, delayed, cpu_count
    import time, math # for display type use ME


    import pylab

    import torchaudio
    from scipy import signal

    import gc
    from path import Path
    
    # Threading
    import threading, queue
    
    return None
    

In [ ]:
def onlyNAmerica(data, data_info=[]): # top 3 are N America, will filter out of data, based on data_info (in case different)
    if len(data_info)<1:
        data_info = data
    top_3 = list(data_info['country'].value_counts().head(3).reset_index()['index'])
    dInfo_top_3 = data_info[data_info['country'].isin(top_3)]
    data_top_3 = data[data['filename'].isin(dInfo_top_3['filename']) ]  #logical_not = data[~data
    return data_top_3

In [ ]:
def add_constants(class_instance):
    class_instance.nSAMPLERATE = 22050
    class_instance.nMAXSONGITER = 5
    return None
    

In [ ]:
def add_variables(class_instance):
    class_instance.bRUNNING = True # was bRUNNING
    class_instance.bSUBMITRUN = False
    class_instance.bPREDICTRUN = False ## Set to False for Data, True for Predict runs (automatically)
    class_instance.bFASTRUN = True
    class_instance.bRUNPARALLEL = True

    class_instance.bFILTER = True # butter filter
    
    return None

In [ ]:
def add_files(class_instance):
    class_instance.bRUNNING = True # was bRUNNING
    class_instance.bSUBMITRUN = False
    class_instance.bPREDICTRUN = False ## Set to False for Data, True for Predict runs (automatically)
    class_instance.bFASTRUN = True
    class_instance.bRUNPARALLEL = True

    class_instance.bFILTER = True # butter filter
    
    
    class_instance.model_filename = 'BFinder_model.sav'  # if first run, copy file to output folder (for path consistency..)
    if os.path.isfile(class_instance.model_filename):
        print("Model File already exists in output.")
    else:
        shutil.copy2('/kaggle/input/zobirdsongdata3/BFinder_model.sav', 'BFinder_model.sav') # target filename is /dst/dir/file.ext
        print("Copying from saved model to output folder")
    # Step on file in output if not saved

    class_instance.TEST = Path("../input/birdsong-recognition/test_audio").exists()
    class_instance.bSUBMITRUN = class_instance.TEST
    if class_instance.TEST:
        class_instance.DATA_DIR = str(Path("../input/birdsong-recognition"))
        print("SUBMIT PATH EXISTS")
    else:
        # dataset created by @shonenkov, thanks!
        class_instance.DATA_DIR = str(Path("../input/birdcall-check"))
        print("NON-SUBMIT - Using Alternative PATH")

    print("DATA_DIR (Dynamic) = ",class_instance.DATA_DIR)


    class_instance.TEST_FOLDER = class_instance.DATA_DIR + '/test_audio/'
    class_instance.test = pd.read_csv(class_instance.DATA_DIR + "/test.csv")

    class_instance.test.head()
    
    
    
    
    
    return None

In [ ]:
class BirdFinder(object):
    # Class Variables (public)
    source_data = []
    
    # Class Constants.  - outside class prefix like: instance.variable
#     bRUNNING = True
    
    def __init__(self):
        self.source_data = []
        self.private_data = [] # class only access
        add_constants(self)
        add_variables(self)
        add_files(self)

    def add(self, x):
        self.source_data.append(x)

    def addtwice(self, x):
        self.add(x)
        self.add(x)
        
    

In [ ]:
app_imports()


# Creates new instance.
my_finder = BirdFinder()
# Calls its attribute.
my_finder.addtwice([5])
print(my_finder.source_data)

In [ ]:
my_finder.nMAXSONGITER